**1. Define the toy dataset**

In [1]:
x = [[1, 2], [3, 4], [5, 6], [7, 8]]
y = [[3], [7], [11], [15]]

**2. Import the relevant packages and define the device we will work on**

In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

**3. Define the Dataset class`(MyDataset)`**

In [3]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
    def __len__(self):
        return len(self.x)

In [4]:
# Define the dataset and dataloader object
ds = MyDataset(x, y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

**4. Define the model architecture using the `Sequential` method available in `nn` package**

In [6]:
model = nn.Sequential(
    nn.Linear(2, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
).to(device)

**5. Print summary of the model**

In [9]:
%pip install torch_summary
from torchsummary import summary

Note: you may need to restart the kernel to use updated packages.


In [14]:
summary(model, torch.zeros(1, 2))

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

**6. Define the loss function and optimizer and train the model**

In [16]:
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(model.parameters(), lr=0.001)
import time
loss_history = []
start = time.time()
for _ in range(50):
    for ix, iy in dl:
        opt.zero_grad()
        loss_value = loss_func(model(ix), iy)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.37045931816101074


**7. Prediction**

In [17]:
# Define the validation dataset
val = [[8,9], [10, 11], [1.5, 2.5]]

In [18]:
model(torch.tensor(val).float().to(device))


tensor([[17.0311],
        [21.0548],
        [ 3.9540]], grad_fn=<AddmmBackward0>)

**Model Saving**

In [20]:
torch.save(model.to('cpu').state_dict(), 'mymodel.pth')

**Loading**

In [21]:
load_path = 'mymodel.pth'
state_dict = torch.load(load_path)
model.load_state_dict(state_dict)

<All keys matched successfully>

**Predictions**

In [22]:
model.to(device)
model(torch.tensor(val).float().to(device))

tensor([[17.0311],
        [21.0548],
        [ 3.9540]], grad_fn=<AddmmBackward0>)